# PlayDB 사이트 이용 - 뮤지컬 작품별 ID 값 추출

In [3]:
!pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [26]:
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm

In [5]:
# 2007년 뮤지컬 url
page_url = "http://www.playdb.co.kr/playdb/playdblist.asp?sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=&sStartYear=2007&sSelectType=1"


header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}

In [6]:
r = requests.get(page_url, headers=header)

In [7]:
soup = BeautifulSoup(r.text,'html.parser')
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=10" http-equiv="X-UA-Compatible"/>
<meta content="_3jpZUBsfY_RI-t71fI9ZxlRRKAs7eA78BXU255TO_g" name="google-site-verification">
<link href="http://ticketimage.interpark.com/TicketImage/playdb/main/favicon.ico" rel="shortcut icon"/>
<link href="//ticketimage.interpark.com/TicketImage/uidev/playDB/styles/common/footer.css" rel="stylesheet"> <!-- RD-1234 푸터변경 -->
<script src="//ticket.interpark.com/common/footer/footer.js" type="text/javascript"></script>
<title>뮤지컬- PLAY DB</title>
<meta content="뮤지컬 - 공연에 대한 모든 것, 플레이DB에서 만나보세요" name="description"/>
<meta content="playdb, 플레이DB, 플레이디비, 공연정보, 뮤지컬, 콘서트, 연극, 클래식, 무용, 전시, 인터파크, 인터파크티켓, 공연랭킹, 인기공연" name="keywords"/>
<link href="http://m.playdb.co.kr/Play/List?maincategory=000001" medi

## 작품별 번호를 가져오는 테스트

In [8]:
# 작품 번호가 담긴 a 태그를 찾기 위해 모든 a 태그 요소 가져오기

a_tags = soup.find_all('a')
a_tags

[<a href="http://www.interpark.com/" target="_blank">인터파크 홈</a>,
 <a href="/member/login.asp">로그인</a>,
 <a href="javascript:ShowServiceLayer('open')">서비스 전체보기</a>,
 <a href="http://ticket.interpark.com/" target="_blank">인터파크 티켓</a>,
 <a href="/Index.asp"><img alt="PlayDB" src="http://ticketimage.interpark.com/TicketImage/playdb/2010_renewal/logo.jpg"/></a>,
 <a href="javascript: HDSearch.Search();" onfocus="this.blur()"><input alt="검색" class="btn" src="http://ticketimage.interpark.com/TicketImage/playdb/2010_renewal/btn_search.gif" type="image"/></a>,
 <a href="/playdb/playdbDetail.asp?sReqPlayno=193169">송가인 콘서트 - 전주</a>,
 <a href="/artistdb/detail.asp?ManNo=29234">김선호</a>,
 <a href="/ranking/TotalRanking.asp"><img alt="금일검색랭킹" src="http://ticketimage.interpark.com/TicketImage/playdb/2010_renewal/ranking_title2.gif"/></a>,
 <a href="javascript:ShowRank_Layer('2');"><img alt="닫기" src="http://ticketimage.interpark.com/TicketImage/playdb/2010_renewal/btn_close.gif"/></a>,
 <a href="/playd

In [9]:
musical_num = []

In [10]:
# a 태그가 특별한 class, id 값이 없기 때문에
# onclick="goDetail" 함수가 내장된 a 태그를 찾고
# 함수 안의 작품번호들만 따로 추출
## 190722의 경우 웹 페이지의 각 연도별 뮤지컬 작품이 아닌 현재 공연 중인 뮤지컬의 대표작에 대한
### 링크이므로 이를 제외, 이것은 현재 공연 중인 작품 리스트에서 추출 가능.
for i in a_tags:
    onclick_attribute = i.get('onclick')
    if onclick_attribute and "goDetail(" in onclick_attribute:
        number = onclick_attribute.split("('")[1].split("')")[0]
        if number == '190722':
            continue
        else:
            if number in musical_num:
                continue
            else:
                musical_num.append(number)

In [11]:
musical_num

['3885',
 '3635',
 '4829',
 '3322',
 '4424',
 '3780',
 '2815',
 '3309',
 '5154',
 '3192',
 '3823',
 '4166',
 '4793',
 '134700']

## 전체 `Musical ID` 가져오기 - 2023년 11월 8일 날짜 기준으로 추출

In [59]:
# 20,21,22년은 코로나 여파로 공연 X / 2007~2019년 사이 연도별 뮤지컬 찾기
years = [2007 ,2008 ,2009 ,2010 ,2011 ,2012 ,2013 ,2014 ,2015 ,2016 ,2017 ,2018 ,2019]

# 2007~2019년 사이 페이지 수
pages = [8 ,59 ,74 ,63 ,95 ,124 ,114 ,103 ,121 ,138 ,133 ,165 ,170]

# 현재공연예정, 개막예정 url 접근 코드
now_codes = [2,3]

# 현재공연예정, 개막예정 항목의 페이지 수
now_pages = [6,25]

# 뮤지컬 번호 저장 리스트
musical_ids=[]

# requests 요청 때 header 값
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}

- 아래 코드에서
```
if id == '190722':
    continue
```
한 이유 ⬇️
#### 190722의 경우 웹 페이지의 각 연도별 뮤지컬 작품이 아닌 현재 공연 중인 뮤지컬의 대표작에 대한
#### 링크이므로 이를 제외, 이것은 현재 공연 중인 작품 리스트에서 추출 가능.

### 2007년~2019년 뮤지컬 ID 추출

In [20]:
for y,p in zip(years, pages):
    print(f"y와 p 값 확인:{y}, {p}")
    
    for i in tqdm(range(1,p+1)):
        page_url = f"http://www.playdb.co.kr/playdb/playdblist.asp?Page={i}&sReqMainCategory=000001\
            &sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=1&sStartYear={y}&sSelectType=1"
        try:
            r = requests.get(page_url, headers=header)
            soup = BeautifulSoup(r.text,'html.parser')
            a_tags = soup.find_all('a')
            
            for j in a_tags:
                onclick_attribute = j.get('onclick')
                if onclick_attribute and "goDetail(" in onclick_attribute:
                    id = onclick_attribute.split("('")[1].split("')")[0]
                    if id == '190722':
                        continue
                    else:
                        if id in musical_ids:
                            continue
                        else:
                            musical_ids.append(id)
            # 페이지마다 3초대기
            time.sleep(3)
            
        # 에러 메세지 처리    
        except requests.exceptions.RequestException as re:
            print(f"Request Error: {re}")
            
        except Exception as e:
            print(f"Other Error: {e}")
            
    print(f"===================={y}년 끝!==========================================================")
    # 연도별 페이지 로드 시 10초 대기
    time.sleep(10)
        
    
        

y와 p 값 확인:2007, 8


100%|██████████| 8/8 [00:28<00:00,  3.57s/it]


====================2007년 끝!==========================================================
y와 p 값 확인:2008, 59


100%|██████████| 59/59 [03:30<00:00,  3.56s/it]


====================2008년 끝!==========================================================
y와 p 값 확인:2009, 74


...

====================2017년 끝!==========================================================
y와 p 값 확인:2018, 165


100%|██████████| 165/165 [09:50<00:00,  3.58s/it]


====================2018년 끝!==========================================================
y와 p 값 확인:2019, 170


100%|██████████| 170/170 [10:10<00:00,  3.59s/it]


====================2019년 끝!==========================================================


In [21]:
len(musical_ids)

20449

### 현재 연도(2023년) 공연 중 및 공연 예정인 뮤지컬 데이터 추출

In [53]:
present_ids = []

In [54]:
for c,p in zip(now_codes, now_pages):
    print(f"c와 p 값 확인:{c}, {p}")
    
    for i in tqdm(range(1,p+1)):
        page_url = f"http://www.playdb.co.kr/playdb/playdblist.asp?Page={i}&sReqMainCategory=000001\
                    &sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType={c}&sStartYear=&sSelectType=1"
        
        r = requests.get(page_url, headers=header)
        soup = BeautifulSoup(r.text,'html.parser')
        a_tags = soup.find_all('a')
        
        for j in a_tags:
            onclick_attribute = j.get('onclick')
            if onclick_attribute and "goDetail(" in onclick_attribute:
                id = onclick_attribute.split("('")[1].split("')")[0]
                print(f"현재 id 값 : {id}")
                if id in present_ids:
                    print(f"{id} already exists")
                    continue
                else:
                    present_ids.append(id)
        # 페이지마다 3초대기

    if c ==2:
        print(f"====================현재 공연 중인 뮤지컬 끝!==========================================================")
    elif c ==3:
        print(f"====================개막예정 뮤지컬 끝!==========================================================")
        
    # 연도별 페이지 로드 시 10초 대기
    time.sleep(10)
        
    
        

c와 p 값 확인:2, 6


 17%|█▋        | 1/6 [00:00<00:01,  3.42it/s]

현재 id 값 : 190722
현재 id 값 : 190036
현재 id 값 : 183052
현재 id 값 : 190718
현재 id 값 : 190723
현재 id 값 : 191013
현재 id 값 : 191968
현재 id 값 : 190726
현재 id 값 : 192992
현재 id 값 : 194463
현재 id 값 : 194225
현재 id 값 : 192343
현재 id 값 : 193854
현재 id 값 : 193404
현재 id 값 : 194470
현재 id 값 : 190256


 33%|███▎      | 2/6 [00:00<00:01,  3.61it/s]

현재 id 값 : 190722
190722 already exists
현재 id 값 : 44865
현재 id 값 : 164746
현재 id 값 : 193773
현재 id 값 : 194902
현재 id 값 : 191356
현재 id 값 : 193702
현재 id 값 : 194005
현재 id 값 : 194189
현재 id 값 : 194661
현재 id 값 : 192173
현재 id 값 : 192425
현재 id 값 : 193975
현재 id 값 : 194688
현재 id 값 : 174075
현재 id 값 : 174445


...

현재 id 값 : 190722
190722 already exists
현재 id 값 : 195837
현재 id 값 : 192506
현재 id 값 : 192633
현재 id 값 : 192646
현재 id 값 : 192996
현재 id 값 : 193183
현재 id 값 : 193193
현재 id 값 : 193458
현재 id 값 : 193850
현재 id 값 : 194277
현재 id 값 : 194310
현재 id 값 : 194325
현재 id 값 : 194362
현재 id 값 : 194464
현재 id 값 : 194649


 96%|█████████▌| 24/25 [00:05<00:00,  3.97it/s]

현재 id 값 : 190722
190722 already exists
현재 id 값 : 194675
현재 id 값 : 194770
현재 id 값 : 194771
현재 id 값 : 194977
현재 id 값 : 195101
현재 id 값 : 195156
현재 id 값 : 195508
현재 id 값 : 195735
현재 id 값 : 195780
현재 id 값 : 195817
현재 id 값 : 195827
현재 id 값 : 195831
현재 id 값 : 195835
현재 id 값 : 190126
현재 id 값 : 192795


100%|██████████| 25/25 [00:06<00:00,  4.13it/s]

현재 id 값 : 190722
190722 already exists
현재 id 값 : 192879
현재 id 값 : 192982
현재 id 값 : 193414
현재 id 값 : 193841
현재 id 값 : 193876
현재 id 값 : 194246
현재 id 값 : 194248
현재 id 값 : 194260
현재 id 값 : 194423
현재 id 값 : 194499
현재 id 값 : 194619
현재 id 값 : 194628
현재 id 값 : 194634
현재 id 값 : 194642
현재 id 값 : 194647
====================개막예정 뮤지컬 끝!==========================================================


In [55]:
len(present_ids)

464

## 현재 공연 중인 뮤지컬 ID 만 가져오기

In [71]:
present_ids = []
print(present_ids)

[]


In [72]:
for i in tqdm(range(1,7)):
    page_url = "http://www.playdb.co.kr/playdb/playdblist.asp?Page={}&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=2&sStartYear=&sSelectType=1".format(i)

    print(page_url)
    r = requests.get(page_url, headers=header)
    soup = BeautifulSoup(r.text,'html.parser')
    a_tags = soup.find_all('a')
    
    for j in a_tags:
        onclick_attribute = j.get('onclick')
        if onclick_attribute and "goDetail(" in onclick_attribute:
            id = onclick_attribute.split("('")[1].split("')")[0]
            # print(f"현재 id 값 : {id}")
            if id in present_ids:
                print(f"{id} already exists")
                continue
            else:
                present_ids.append(id)
    # 페이지마다 3초대기
# 연도별 페이지 로드 시 10초 대기
time.sleep(10)
    

    

  0%|          | 0/6 [00:00<?, ?it/s]

http://www.playdb.co.kr/playdb/playdblist.asp?Page=1&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=2&sStartYear=&sSelectType=1


 17%|█▋        | 1/6 [00:00<00:01,  3.02it/s]

http://www.playdb.co.kr/playdb/playdblist.asp?Page=2&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=2&sStartYear=&sSelectType=1


 33%|███▎      | 2/6 [00:00<00:01,  3.38it/s]

...

190722 already exists
http://www.playdb.co.kr/playdb/playdblist.asp?Page=5&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=2&sStartYear=&sSelectType=1


 83%|████████▎ | 5/6 [00:01<00:00,  3.56it/s]

190722 already exists
http://www.playdb.co.kr/playdb/playdblist.asp?Page=6&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=2&sStartYear=&sSelectType=1


100%|██████████| 6/6 [00:01<00:00,  3.46it/s]

190722 already exists
193057 already exists


In [68]:
len(present_ids)

90

## "공연 예정" 인 뮤지컬 ID 값만 가져오기

In [77]:
future_ids = []

In [78]:
for i in tqdm(range(1,28)):
    page_url = f"http://www.playdb.co.kr/playdb/playdblist.asp?Page={i}&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=3&sStartYear=&sSelectType=1"

    print(page_url)
    
    r = requests.get(page_url, headers=header)
    soup = BeautifulSoup(r.text,'html.parser')
    a_tags = soup.find_all('a')
    
    for j in a_tags:
        onclick_attribute = j.get('onclick')
        if onclick_attribute and "goDetail(" in onclick_attribute:
            id = onclick_attribute.split("('")[1].split("')")[0]
            # print(f"현재 id 값 : {id}")
            if id in future_ids:
                print(f"{id} already exists")
                continue
            else:
                future_ids.append(id)
    # 페이지마다 3초대기
# 연도별 페이지 로드 시 10초 대기
time.sleep(10)
    

    

  0%|          | 0/27 [00:00<?, ?it/s]

http://www.playdb.co.kr/playdb/playdblist.asp?Page=1&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=3&sStartYear=&sSelectType=1


  4%|▎         | 1/27 [00:00<00:05,  4.53it/s]

190722 already exists
http://www.playdb.co.kr/playdb/playdblist.asp?Page=2&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=3&sStartYear=&sSelectType=1


  7%|▋         | 2/27 [00:00<00:06,  4.03it/s]

...

190722 already exists
http://www.playdb.co.kr/playdb/playdblist.asp?Page=25&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=3&sStartYear=&sSelectType=1
190722 already exists
http://www.playdb.co.kr/playdb/playdblist.asp?Page=26&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=3&sStartYear=&sSelectType=1


 96%|█████████▋| 26/27 [00:06<00:00,  4.13it/s]

190722 already exists
http://www.playdb.co.kr/playdb/playdblist.asp?Page=27&sReqMainCategory=000001&sReqSubCategory=&sReqDistrict=&sReqTab=2&sPlayType=3&sStartYear=&sSelectType=1


100%|██████████| 27/27 [00:06<00:00,  4.21it/s]

190722 already exists
195798 already exists
195842 already exists
195852 already exists
195854 already exists
195857 already exists
195858 already exists
195859 already exists
194240 already exists


In [79]:
len(future_ids)

397

In [80]:
future_ids.remove("190722")

In [81]:
len(future_ids)

396

## 저장된 뮤지컬 id 리스트 pickle 파일로 저장 -> 다른 노트북 파일에서 사용

In [2]:
import pickle

In [25]:
# musical_id.pkl에 저장

with open('musical_id.pkl', 'wb') as f:
    pickle.dump(musical_ids, f)

In [3]:
# id 값 불러오기
with open("musical_id.pkl", "rb") as file:
    loaded_musical_id = pickle.load(file)

In [31]:
len(loaded_musical_id)

20477

In [44]:
search_value = '153118'

for index, value in enumerate(loaded_musical_id):
    if value == search_value:
        print(f"{search_value}은 리스트의 {index}번째 위치에 있습니다.")
        break
else:
    print(f"{search_value}은 리스트 안에 없습니다.")

153118은 리스트의 20448번째 위치에 있습니다.


## 2007년부터 2019년 까지의 뮤지컬 ID 값만 가져와 저장

In [38]:
# id 값 불러오기
with open("musical_id.pkl", "rb") as file:
    loaded_musical_id = pickle.load(file)

In [46]:
train_musical_id = loaded_musical_id[:-28]
len(train_musical_id)

20449

In [47]:
with open('train_musical_id.pkl', 'wb') as f:
    pickle.dump(train_musical_id, f)

In [58]:
with open("train_musical_id.pkl", "rb") as file:
    train = pickle.load(file)
train[-1]

'153118'

### "현재 공연 중인 뮤지컬" ID 값만 가져와 따로 저장

In [69]:
with open('present_musical_id.pkl', 'wb') as f:
    pickle.dump(present_ids, f)

In [70]:
with open("present_musical_id.pkl", "rb") as file:
    present = pickle.load(file)
present[1:5]

['190036', '183052', '190718', '190723']

### "공연 예정 뮤지컬" ID 값만 가져와 따로 저장 

In [82]:
with open('future_musical_id.pkl', 'wb') as f:
    pickle.dump(future_ids, f)

In [83]:
with open("future_musical_id.pkl", "rb") as file:
    future = pickle.load(file)
future[1:5]

['194066', '193886', '193988', '195191']